# Import Libraries

In [4]:
import pandas as pd
import numpy as np
import re
import gensim
from gensim import corpora
import spacy
import aspose.words as aw
from spacy.tokenizer import Tokenizer

# Define Preprocessing Functions

In [16]:
# load pre-trained model
nlp = spacy.load('en_core_web_sm')
# Tokenize words only with the whitespace rule
# N-grams will no longer be treated as 'N' and '-grams'
nlp.tokenizer = Tokenizer(nlp.vocab, token_match=re.compile(r'\S+').match)

def preprocess(txt):
    txt = txt.lower()
    
    # these must come first
    txt = re.sub('b\.\S*', '', txt) # remove all bachelor qualifications
    txt = re.sub('m\.\S*', '', txt) # remove all master qualifications


    txt = re.sub("'", "", txt) # Remove apostrophe
    txt = re.sub("’", "", txt) # Remove apostrophe
    txt = re.sub('http\S+\s*', ' ', txt)  # remove URLs
    txt = re.sub('RT|cc', ' ', txt)  # remove RT and cc
    txt = re.sub('#\S+', '', txt)  # remove hashtags
    txt = re.sub('@\S+', ' ', txt)  # remove mentions
    txt = re.sub('[^a-zA-Z]', ' ', txt) # Remove non-English charcters
    txt = re.sub('\s+', ' ', txt)  # remove extra whitespace

    # tokenize word
    txt = nlp(txt)

    # remove stop words and lemmatization
    txt = [token.text for token in txt if not token.is_stop]

    return ' '.join(txt)

# STOP = nltk.corpus.stopwords.words('english')

# Suggestion:
# def topic_modelling(resume_text, num_words=10, min_prob = 10**-2):
#     doc_clean = [resume_text.split()]   
    
#     # term dictionary
#     dictionary = corpora.Dictionary(doc_clean)

#     # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
#     doc_term_mat = [dictionary.doc2bow(doc) for doc in doc_clean]

#     # latent dirichlet allocation model 
#     Lda = gensim.models.ldamodel.LdaModel

#     ldamodel = Lda(doc_term_mat, num_topics=1, id2word = dictionary, passes=50)
    
#     # Return only the topic words that have the probability of larger than .01
#     return [token for token, prob in ldamodel.show_topic(0, topn=num_words) if prob > min_prob ]

def n_grams(tokens, n):
    return [' '.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

# Load all the skills from EMSI Skills API dataset

In [20]:
skills = pd.read_excel("all_skills_emsi.xlsx")
# remove all the additional descriptions in round brackets
skills['name'] = skills['name'].apply(lambda x: re.sub("\W\(.*?\)","",x))
skills['name'] = skills['name'].apply(lambda x: x.lower())
skills['lemmatized_name'] = skills['name'].apply(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))
skills.head()

,Unnamed: 0,id,infoUrl,name,type.id,type.name,lemmatized_name
0,0,KS126XS6CQCFGC3NG79X,https://skills.emsidata.com/skills/KS126XS6CQC...,.net assemblies,ST1,Specialized Skill,.net assembly
1,1,ES50D03AC9CFC1A0BC93,https://skills.emsidata.com/skills/ES50D03AC9C...,.net development,ST1,Specialized Skill,.net development
2,2,KS1200B62W5ZF38RJ7TD,https://skills.emsidata.com/skills/KS1200B62W5...,.net framework,ST1,Specialized Skill,.net framework
3,3,KS126XW78QJCF4TRV2X7,https://skills.emsidata.com/skills/KS126XW78QJ...,.net framework 1,ST1,Specialized Skill,.net framework 1
4,4,KS126XY68BNKXSBSLPYS,https://skills.emsidata.com/skills/KS126XY68BN...,.net framework 3,ST1,Specialized Skill,.net framework 3


In [21]:
# Convert all the skills into a big hashset
skills_api = set(skills['name']).union(set(skills['lemmatized_name']))

In [22]:
def extract_skills(resume_text, skills_api, clean=True):
    if clean == True:
        resume_text = preprocess(resume_text)
        
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    # all the resume skills will be saved here
    skillset = []
    
    # check for one-grams (example: python)
    for token in tokens:
        skillset.append(token)
    
    # check for noun_chunks (example: machine learning)
    for token in nlp_text.noun_chunks:
        token = token.text.lower().strip()
        skillset.append(token)

    # check for N-grams that SpaCy missed in the noun_chuncks
    for n in range(2, 10):
        for token in n_grams(tokens, n):
            token = token.lower().strip()
            skillset.append(token)
    
    # get only the unique skills in lowercase
    skillset = set([i for i in set([i.lower() for i in skillset])])

    return skillset.intersection(skills_api)

# Read any resume

In [33]:
resume = aw.Document("31605080.pdf")
resume_string = resume.to_string(aw.SaveFormat.TEXT).split('\r\n')
resume_string = ' '.join(resume_string[1:-3])

In [34]:
resume_string[:1000]

'GEEK SQUAD AGENT Professional Profile IT support specialist with experience across multiple disciplines including technical support, customer service, computer repair, and military service. I am hard working, willing to learn, team oriented, and comfortable working independently as well. Qualifications Windows / Mac / IOS / Android Technical Support Hardware & Software Maintenance User Training Malware Detection & Removal Customer Service Entry Level Active Directory & Ticketing Problem Solving & Research Experience Company Name August 2014 to October 2016 Geek Squad Agent City , State · Provided technical support in person and over the phone. · Performed hardware and software installation and repair. · Refurbished and setup PCs and peripheral devices. Company Name January 2013 to January 2014 Shipping & Receiving Associate City , State · Performed shipping and receiving of product. · Assisted with inventory management. · General logistics and warehouse duties. Company Name January 20

# Extract Skills

In [35]:
extract_skills(resume_text=resume_string, skills_api=skills_api, clean=True)

{'active directory',
 'android',
 'aviation',
 'computer repair',
 'customer service',
 'information technology',
 'installation',
 'inventory management',
 'level',
 'logistics',
 'malware detection',
 'management',
 'marshalling',
 'military service',
 'peripheral devices',
 'problem solving',
 'profile',
 'repair',
 'research',
 'research experience',
 'safe',
 'software installation',
 'software maintenance',
 'team oriented',
 'technical support',
 'troubleshooting',
 'warehouse'}

However, this is actually assuming all the skills have the same level of experience.